# How to renormalize the Schrödinger equation

## Introduction

In the following notebook I want to repeat the analysis of Lepage described in the paper https://arxiv.org/abs/nucl-th/9706029. The main idea is that we can apply the idea behind renormalization of field theories - low-energy approximation to arbitrary high-energy physics - even in the context of non-relativistic standard Schrödinger equation. The transformation from the real theory to a simpler effective theory is achieved in two steps. First we introduces a cutoff $\Lambda$ that is of order of the momentum at which unkown physics becomes important. Obviously we don't know the scale $\Lambda$ at which this happens but results are almost independent of $\Lambda$ provided it is much larger then the momenta in the range being probed experimentally. The second step is to add local interactions mimicking the effects of the true short-distance physics. The reason lies in the uncertainty principle: any state involving momentum larger then the cutoff is necessary highly virtual and must occur over distances of order $1/\Lambda$ or less. In principle there are infinitley manu correction term but, when whorking to a given precision, only a finite number of them are important.
At the end only it is only the numerical values of the couplings that contain information about the short-range physics.
Before illustrating the construction and use of an effective theory that reproduces a given collection of low-energy data, I discuss the data which are generated inventing a physical system and solving the Schrödinger equation that describes it.

## Synthetic data solving Schrödinger equation
For the system we can consider the familiar one-particle Coulombic atom, but with a short-range potentiale $V_S(r)$ in addition to the Coulomb potential:
$$
H = \frac{\textbf{p}^2}{2m} + V(r)
$$
where
$$
V(r) = - \frac{Zq_e^2}{r} + V_S(r).
$$
For our purposes the form of $V_S(r)$ is irrelevant, without any particular reason I chose
$$
V_S(r) = b_0 \frac{e^{- r / b_1}}{r},
$$
where $b_0, b_1$ are parameters determining the precise form of the potential. Once we have the potential we can solve the radial Schrödinger equation
$$
-\frac{\hbar}{2m} \frac{d^2 \chi}{dr^2} + \left[V(r) + \frac{\hbar^2 l (l +1)}{2mr^2} - E \right] \chi(r) = 0
$$
To solve numerically this equation I used Numerov's method.

### Numerov's method
Numerov's method (after the Russian astronomer Boris Vasilyevich Numerov) is useful to integrate second-order differential equations of the general form
$$
\frac{d^2 y}{dx^2} = - g(x) y(x) + s(x).
$$
Taylor expanding $y(x)$ backward and forward up to fifth order and then taking the sum we get
$$
y_{n+1} + y_{n_1} = 2y_n + y_n^"(\Delta x)^2 + \frac{1}{12}y_n^{""}(\Delta x)^4 + \mathcal{O}[(\Delta x)^6]
$$
One can define $y_n^" \coloneqq z_n$, and repeat Taylor expanding as above to obtain
$$
y_n^{""} = z_n^" = \frac{z_{n+1} + z_{n-1} - 2z_n}{(\Delta x)^2} + \mathcal{O}[(\Delta x)^2].
$$
We can substitute above and obtain the $\text{\emph{Numerov's formula}}$
$$
y_{n+1}\left[1 + g_{n+1} \frac{(\Delta x)^2}{12}\right] = 2y_n \left [1 - 5g_n \frac{(\Delta x)^2}{12} \right] - y_{n-1}\left[1 + g_{n-1} \frac{(\Delta x)^2}{12}\right] + (s_{n +1} +10 s_n + s_{n -1})\frac{(\Delta x)^2}{12} + \mathcal{O}[(\Delta x)^6]
$$
In the case of Schrödigner equation $s-$terms are absent and it it convenient to define $f_n \coloneqq 1 + g_n (\Delta x)^2 / 12$ and rewrite
$$
y_{n+1} = \frac{(12-10f_n)y_n + f_{n-1}y_{n-1}}{f_{n+1}}.
$$
To avoid the problem of divergence at large x the matching method is implemented, so actually two integrations are performed: a forward recursion starting from $0$ and a backward one starting from $x_{\text{max}}$. The matching point is chosen in correspondence of the classical inversion point $x_{\text{cl}}$, i.e. where $V(x_{\text{cl}})$. Before starting the analysis we need to notice another critical aspect of trying to solve straightforwardly the radial equation: the singularity of the potential at $r = 0$.

### Logarithmic grid
One way to avoide the problem of singularity is to work with a variable-step grid instead of a constant-step one. We can introduce a variable $x$ and a constant-step grid in $x$ sp that we are able to use Numerov's method without changes. Then we define a mapping between $r$ and $x$
$$
x(r) = \text{log}\frac{Zr}{a_0}
$$
where $a_0$ is the Bohr radius. This choice yields
$$
\Delta x = \frac{\Delta r}{r},
$$
the ratio $\Delta r / r$ remains constant on the grid of $r$, called $\text{\emph{logarithmic gric}}$. Unfortunately transforming the radial equation using variable $x$ a term with the first derivative appears preventing the usage of Numerov's. This can be solved by the following transformation
$$
y(x) = \frac{1}{\sqrt{r}}\chi(r(x)),
$$
substituting and multiplying by $r^{3/2}$ we get
$$
\frac{d^2 y}{dx^2} + \left[ \frac{2m_e}{\hbar^2} r^2 (E - V(r)) - \left(l + \frac{1}{2}\right)^2\right]y(x) = 0
$$
which is solvable by Numerov with
$$
g(x) = \frac{2m_e}{\hbar^2}r(x)^2 (E-V(r)) - \left( l + \frac{1}{2} \right)^2.
$$
It is practical to work with $\text{\emph{atomic units}}$ (a.u.): units of length will be expressend in Bohr radii $a_0$:
$$
a_0 = \frac{\hbar^2}{m_e q_e^2} = 0.529177 \times 10^{-10} \text{m},
$$
while energies are expressed in $\text{\emph{Rydberg}}$ (Ry):
$$
1 \text{Ry} = \frac{m_e q_e^4}{2 \hbar^2} = 13.6058 \text{eV}.
$$
In such units we have $\hbar = 1$, $m_e = 1/2$ and $q_e^2 = 2$.
Creation of the logarithmic gris is the starting point of our program, this is done by the function "prepare_grid" which defines once and for all the values of $r$, $\sqrt{r}$, $r^2$ for each grid point. We have

In [1]:
#Radial Schrodinger equation using Numerov method

import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from scipy.optimize import curve_fit


def prepare_grid(n_points, x_min, dx):

    #preparing x-array with constant step
    x = np.linspace(x_min, x_min + ((n_points - 1) * dx), n_points)
    x = np.append(x, x[n_points - 1] + dx) #another point for convenience of indices

    #generate r, sqrt_r, and r^2 based on logarithmic x grid
    r = np.exp(x)
    sqrt_r = np.sqrt(r)
    r2 = np.power(r, 2)

    #print grid information
    print("Radial grid information:\n")
    print("dx = ", dx)
    print("x_min = ", x_min)
    print("n_points = ", n_points)
    print("r(0) = ", r[0])
    print("r(n_points) = ", r[n_points])
    print("-----------------------------------------------\n")

    return r, sqrt_r, r2

### Potential
To set the potential we will use the function "init_potential" which has another suffix to specify the specific potential, i.e. "init_potential_coulomb".
We will consider first the simple Coulomb potential to test the code and the we will consider the short term.

In [2]:
def init_potential_coulomb(r):

    #definition of the potential
    v_pot = -2/r #(Rydberg) atomic units

    #saving the potential to CSV
    df_pot = pd.DataFrame({"r": r, "V(r)": v_pot})
    df_pot.to_csv("coulomb_potential.csv", index=False)
    print("potential saved in 'coulomb_potential.csv'\n")

    
    #plotting the potential
    plt.figure()
    plt.plot (r, v_pot, label=r'V(r), atomic units (Ry)', color ='b')
    plt.xlabel("r")
    plt.ylabel("V(r)")
    plt.title("coulomb potential")
    plt.grid(True)
    plt.legend()
    plt.xlim(0, 0.01)
    plt.savefig("coulomb_potential_plot.pdf")
    
    return v_pot

def init_potential_short(r):

    #definition of the potential
    v_pot = -2/r - (2 * np.exp(-r/0.1))  / r 

    #saving the potential to CSV
    df_pot = pd.DataFrame({"r": r, "V(r)": v_pot})
    df_pot.to_csv("short_potential.csv", index=False)
    print("potential saved in 'short_potential.csv'\n")

    
    #plotting the potential
    plt.figure()
    plt.plot (r, v_pot, label=r'V(r), atomic units (Ry)', color ='b')
    plt.xlabel("r")
    plt.ylabel("V(r)")
    plt.title("potential with short range term")
    plt.grid(True)
    plt.legend()
    plt.xlim(0, 0.01)
    plt.savefig("short_potential_plot.pdf")

### solve_schrodinger function
The function "solve_schrodinger" is the main function of the program